# CNN for dogs and cats classfication using transfer learning (RESNET50)

## We import the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50 #We import the Resnet50 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#We check the tf version
tf.__version__

'2.12.0'

#### I define the route of the images since I am leaving the code in the root folder thats why I dont use absolutes routes

In [2]:
train_dir = 'training_set'
test_dir = 'test_set'

### Preprocessing the data

In [3]:
#We define the image size of resize and the batch size 
image_size = (224, 224)
batch_size = 32
#We create de image datagenerator specifing the preprocessing_function of the resnet50
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input, validation_split=0.2)  # 20% del conjunto de entrenamiento será utilizado como validación)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

### Creation of data generators

In [4]:
#We create de generators using thr datagens where the class_mode is binary since the results can only be of two types
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)
#Validation is the 20% of the train datagen
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


### CREATE THE MODEL

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
#We define the resnet model removing the top full connected layers because we have more flexibility in the architecture and we can create our custom model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# We create the model
x = base_model.output #We take the output of the last predefine resnet layer
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Crear el modelo final tomando en cuenta la base del modelo y el output previamente definido
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo con la funcion adam
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Imprimir un resumen de la arquitectura del modelo
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              